# Movielens 영화 추천 실습

### 1. 데이터 준비와 전처리

In [1]:
import os
import pandas as pd
import numpy as np

# rating.dat 에 인덱싱까지 완료된 사용자-영화-평점데이터가 깔끔하게 정리
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


### 2) 분석하기
- rating에 있는 유니크한 사용자 수
- ratings에 있는 유니크한 영화 개수
- 가장 인기 있는 영화 30개(인기순)

In [6]:
# 사용자 수
ratings['user_id'].nunique()
print ('유니크한 사용자는 {} 명 이다.'.format(ratings['user_id'].nunique()))

유니크한 사용자는 6039 명 이다.


In [7]:
# 영화 개수
ratings['movie_id'].nunique()
print ('유니크한 영화는 {} 개 이다.'.format(ratings['movie_id'].nunique()))

유니크한 영화는 3628 개 이다.


In [8]:
## rating과 movies를 합친다.
movie_count = pd.merge(ratings, movies, how="left", on="movie_id")

## rating을 count를 기준으로 내림차순해준다.
movie_count.sort_values(by="count", ascending=False, inplace=True)
movie_count.reset_index(inplace=True, drop=True)

## 평가 높은 상위 30개 영화 리스트
movie_count.head(30)

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2339,1952,5,974420466,Midnight Cowboy (1969),Drama
2,4671,1210,5,963794984,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
3,2340,715,5,974421414,"Horseman on the Roof, The (Hussard sur le toit...",Drama
4,2340,2987,5,974422967,Who Framed Roger Rabbit? (1988),Adventure|Animation|Film-Noir
5,4671,380,5,963795314,True Lies (1994),Action|Adventure|Comedy|Romance
6,1293,524,5,974788401,Rudy (1993),Drama
7,1293,527,5,974788915,Schindler's List (1993),Drama|War
8,2339,3929,5,974421058,"Bank Dick, The (1940)",Comedy
9,2339,3922,5,974421095,Bikini Beach (1964),Comedy


[실패]
- movie_count = rationgs.groupby('title')['user_id'].count()  #첫 시도.
- movie_count = count.groupby('title')['movie_id'].count() #rationgs 컬럼 count 변경확인후 재시도.
- movie_count = rationgs.groupby('ratings')['movie_id'].count() #title 대신 ratings으로 재시도.
- movie_count = count.groupby('title')['ratings'].count() #평점 기준 확인 재시도.


   *kalina007님 깃허브 참고 하여 재진행..추후 노드 스터디 복습예정

In [9]:
## 사용할 컬럼 노출
using_cols = ['user_id', 'title', 'count']
data = movie_count[using_cols]
data.head(10)

,user_id,title,count
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2339,Midnight Cowboy (1969),5
2,4671,Star Wars: Episode VI - Return of the Jedi (1983),5
3,2340,"Horseman on the Roof, The (Hussard sur le toit...",5
4,2340,Who Framed Roger Rabbit? (1988),5
5,4671,True Lies (1994),5
6,1293,Rudy (1993),5
7,1293,Schindler's List (1993),5
8,2339,"Bank Dick, The (1940)",5
9,2339,Bikini Beach (1964),5


In [10]:
# 쉬운 검색을 위해 문자열 소문자로 변경

data['title'] = data['title'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,user_id,title,count
0,1,one flew over the cuckoo's nest (1975),5
1,2339,midnight cowboy (1969),5
2,4671,star wars: episode vi - return of the jedi (1983),5
3,2340,"horseman on the roof, the (hussard sur le toit...",5
4,2340,who framed roger rabbit? (1988),5
5,4671,true lies (1994),5
6,1293,rudy (1993),5
7,1293,schindler's list (1993),5
8,2339,"bank dick, the (1940)",5
9,2339,bikini beach (1964),5


### 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가


In [11]:
## user_id 추가
data["user_id"].max()

6040

In [12]:
# 모델 검증을 위한 사용자 초기 정보 세팅

my_favorite = ['farinelli' , 'once' ,'shine' ,'the greatest showman' ,'the lord of the rings'] #소문자 별도처리

# user_id 'sanghee'지정후, 5개 영화의 평점을 4점 줬다고 가정
my_playlist = pd.DataFrame({'user_id': ['sanghee']*5, 'title': my_favorite, 'count':[4]*5})

if not data.isin({'user_id':['sanghee']})['user_id'].any():  # user_id에 'sanghee'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,title,count
836473,4194,james and the giant peach (1996),3
836474,1727,never been kissed (1999),3
836475,1727,jury duty (1995),3
836476,4194,"opposite of sex, the (1998)",3
836477,3080,keeping the faith (2000),3
0,sanghee,farinelli,4
1,sanghee,once,4
2,sanghee,shine,4
3,sanghee,the greatest showman,4
4,sanghee,the lord of the rings,4


In [13]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)} #enumerate()  리스트가 있는 경우 순서와 리스트의 값을 전달하는 기능
title_to_idx = {v:k for k,v in enumerate(title_unique)} 

In [14]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['sanghee'])  
print(title_to_idx['once'])

6039
3629


In [15]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 

temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,count
0,0,0,5
1,1,1,5
2,2,2,5
3,3,3,5
4,3,4,5
...,...,...,...
0,6039,3628,4
1,6039,3629,4
2,6039,3630,4
3,6039,3631,4


[시도] 
- 모든 row가 정상적으로 인덱싱 -> data['user_id']을 인덱싱된 Series로 교체됨 > user_id & title 숫자 변경

### 4) CSR matrix를 직접 만들어 봅시다.

In [16]:
# 사용자의 명시적/암묵적 평가
# 3점만 평가한 데이터의 비율을 보는 코드, 임의 규칙을 정해본다.

only_one = data[data['count']<4] #기준 count
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

261197,836483
Ratio of only_one over all data is 31.23%


[시도]
- only_one = data[data['count']<3]  : 0,836483 ,Ratio of only_one over all data is 0.00%
- only_one = data[data['count']<4]  : 261197,836483, Ratio of only_one over all data is 31.23%
- only_one = data[data['count']<5]  : 610168,836483, Ratio of only_one over all data is 72.94%
- user_id의 '6041'의 평점을 5점에서 ->4점으로 조종하니 72.94% ->72.95%로 변경됨

평점 ~2점 0%, 평점 3점이상 31.23% 평점 4점이상 72.94%로 확인된다. "평점 테러가 적다!!.."

In [17]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

## csr matrix로 만들기
csr_data = csr_matrix((data["count"], (data.user_id, data.title)), shape=(num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

#### MF 모델 학습
- implicit 패키지 (https://github.com/benfred/implicit) : 암묵적(implicit) dataset을 사용하는 다양한 모델을 빠르게 학습할 수 있는 패키지
- 이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용
- Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있습니다.

In [18]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [19]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [20]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [21]:
# 모델 훈련 마무리
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [22]:
# 유저 sanghee 벡터와 영화 once의 벡터를 어떻게 만들고 있는지 확인
sanghee, once = user_to_idx['sanghee'], title_to_idx['once']
sanghee_vector, once_vector = als_model.user_factors[sanghee], als_model.item_factors[once]

print('슝=3')

슝=3


In [23]:
sanghee_vector

array([-7.06410641e-03,  2.39276863e-03,  7.54039735e-03,  1.16030155e-02,
       -1.46426642e-02, -8.85095261e-03,  3.67467850e-03, -7.93435052e-03,
        4.98763518e-04, -1.30738912e-03,  8.06264766e-03, -6.42644567e-03,
        1.78777352e-02,  1.49238051e-03, -1.17647126e-02,  2.33103596e-02,
        3.44832893e-03, -8.15493800e-03,  8.35443381e-03, -7.67456647e-03,
        2.06502117e-02, -8.58852174e-03, -5.51335746e-03, -3.71157657e-03,
       -2.16575642e-03, -2.33851187e-03,  5.14490949e-03,  1.91094186e-02,
       -2.26201732e-02,  7.96257984e-03,  3.56966443e-02,  7.19417119e-03,
       -9.38927103e-03,  2.08385615e-03,  4.28535137e-03, -1.46299293e-02,
       -1.25419796e-02,  8.85775007e-05,  6.96088281e-03, -1.62200499e-02,
       -1.51904151e-02,  6.25127635e-04,  8.71774275e-03,  5.44861052e-03,
       -6.25271723e-03,  1.55426962e-02,  6.72269054e-03, -1.34575367e-02,
        1.25445435e-02,  3.96936899e-03,  1.60308518e-02, -9.75839049e-03,
       -1.26318792e-02,  

In [24]:
once_vector

array([ 1.8423905e-04,  3.2236820e-04,  5.0456770e-04,  5.8117829e-04,
        2.2424910e-04,  2.6428663e-05,  4.9216562e-04,  3.1721874e-04,
        4.7927405e-04,  6.1077514e-04,  5.3691532e-04,  1.8343785e-04,
        5.4780254e-04,  3.9630703e-04,  1.7462940e-04,  7.0942013e-04,
        1.7530819e-04,  1.7584191e-04,  5.5418897e-04,  3.1218215e-04,
        6.3261809e-04,  2.5305682e-04,  1.2142119e-04,  2.2861737e-04,
        4.2710433e-04,  1.9107436e-04,  5.7737221e-04,  9.6334808e-04,
       -5.8401842e-05,  5.8122212e-04,  9.4931707e-04,  5.4398779e-04,
        1.2104994e-04,  4.2869689e-04,  4.1598911e-04, -9.6549365e-06,
        4.3536264e-05,  4.4522795e-04,  6.2462303e-04,  1.8271708e-04,
        1.7651306e-04,  1.1710084e-04,  6.2969996e-04,  4.7170409e-04,
       -1.1210641e-04,  7.9803780e-04,  3.5358054e-04,  1.2959582e-04,
        6.7419861e-04,  3.8993102e-04,  5.7350472e-04,  1.2517543e-04,
        2.2437124e-04,  7.0752506e-04,  5.6393922e-04,  7.5495511e-04,
      

In [25]:
# sanghee와 once를 내적하는 코드
np.dot(sanghee_vector, once_vector)

0.00033865037

In [26]:
# 모델이 나의 shine 영화에 대한 선호도를 어떻게 예측하는지 보자!
shine = title_to_idx['shine']
shine_vector = als_model.item_factors[shine]
np.dot(sanghee_vector, shine_vector)

0.0002724787

In [27]:
# 모델이 나의 farinelli 영화에 대한 선호도를 어떻게 예측하는지 보자!
farinelli = title_to_idx['farinelli']
farinelli_vector = als_model.item_factors[farinelli]
np.dot(sanghee_vector, farinelli_vector)

0.00027939214

In [28]:
# 모델이 나의 the greatest showman 영화에 대한 선호도를 어떻게 예측하는지 보자!
the_greatest_showman = title_to_idx['the greatest showman']
the_greatest_showman_vector = als_model.item_factors[the_greatest_showman]
np.dot(sanghee_vector, the_greatest_showman_vector)

0.0002892859

[시도] 
- the greatest showman 등 영화명이 긴 경우 
- title_to_idx['the greatest showman'] 영화명 그대로 기입 한다.
- als_model.item_factors[the_greatest_showman] 에 <mark>언더바를 붙여야</mark> 오류가 나지 않는다. 

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요

- AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 영화 찾기

In [29]:
favorite_title = 'once'
title_id = title_to_idx[favorite_title]
similar_title = als_model.similar_items(title_id, N=15)
similar_title

[(3629, 0.9999998),
 (3631, 0.9984901),
 (3632, 0.99838585),
 (3628, 0.99632126),
 (3630, 0.990641),
 (3621, 0.9784523),
 (3625, 0.97632176),
 (3620, 0.97317314),
 (3416, 0.9726322),
 (3407, 0.97164226),
 (3624, 0.97153133),
 (3417, 0.9708441),
 (3622, 0.9696872),
 (3404, 0.96944225),
 (3420, 0.96943235)]

In [30]:
# title_to_idx 를 뒤집어, index로부터 title_ 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_title]

['once',
 'the greatest showman',
 'the lord of the rings',
 'farinelli',
 'shine',
 'century (1993)',
 'last of the high kings, the (a.k.a. summer fling) (1996)',
 'war at home, the (1996)',
 "another man's poison (1952)",
 "i don't want to talk about it (de eso no se habla) (1993)",
 'number seventeen (1932)',
 'daens (1992)',
 "brother's kiss, a (1997)",
 'male and female (1919)',
 'neon bible, the (1995)']

In [31]:
def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

print("슝=3")

슝=3


In [32]:
get_similar_title('once')

['once',
 'the greatest showman',
 'the lord of the rings',
 'farinelli',
 'shine',
 'century (1993)',
 'last of the high kings, the (a.k.a. summer fling) (1996)',
 'war at home, the (1996)',
 "another man's poison (1952)",
 "i don't want to talk about it (de eso no se habla) (1993)"]

In [39]:
get_similar_title('jury duty (1995)')

['jury duty (1995)',
 'bio-dome (1996)',
 "national lampoon's senior trip (1995)",
 'almost heroes (1998)',
 'in the army now (1994)',
 'major payne (1994)',
 'senseless (1998)',
 'beverly hillbillies, the (1993)',
 'meet wally sparks (1997)',
 'house arrest (1996)']

- jury duty (1995) 처럼  영화 title에 나와있는 연도를 같이 입력해야 오류가 나지 않았다.

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.
- AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 추천받기

In [40]:
user = user_to_idx['sanghee']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
title_recommended

[(201, 0.005202425),
 (112, 0.0041479394),
 (2944, 0.004036421),
 (903, 0.00403239),
 (941, 0.003979883),
 (1832, 0.0038826996),
 (1689, 0.0038427643),
 (1642, 0.0037763957),
 (1945, 0.0037159328),
 (2555, 0.0037061577),
 (2741, 0.0036791135),
 (2410, 0.0036393022),
 (53, 0.0036045401),
 (1459, 0.003603512),
 (73, 0.003578944),
 (2384, 0.003575331),
 (2082, 0.0035083618),
 (1423, 0.0034885597),
 (131, 0.0034760039),
 (683, 0.0034509879)]

In [41]:
# title_to_idx 를 뒤집어, index로부터 title_ 이름을 얻는 dict를 생성합니다. 
[idx_to_title[i[0]] for i in title_recommended]

['sixth sense, the (1999)',
 'l.a. confidential (1997)',
 'thieves (voleurs, les) (1996)',
 'beautiful thing (1996)',
 'drunken master (zui quan) (1979)',
 'pillow book, the (1995)',
 'jude (1996)',
 'children of heaven, the (bacheha-ye aseman) (1997)',
 'great white hype, the (1996)',
 'before and after (1996)',
 'return of the fly (1959)',
 'hollow reed (1996)',
 'back to the future (1985)',
 'sanjuro (1962)',
 'shakespeare in love (1998)',
 'wild reeds (1994)',
 'alphaville (1965)',
 'when night is falling (1995)',
 'fight club (1999)',
 'blues brothers 2000 (1998)']

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [42]:
# 영화 once 입력했을때
once = title_to_idx['once']
explain = als_model.explain(user, csr_data, itemid=once)

In [44]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('the lord of the rings', 7.25777602095248e-05),
 ('once', 5.9433024955571665e-05),
 ('the greatest showman', 5.053282804658249e-05),
 ('farinelli', 4.860932357362135e-05),
 ('shine', 4.704103556680748e-05)]

In [45]:
# 추천 받은 리스트중 임의로 sixth sense, the (1999) 을 입력 했을때
once = title_to_idx['sixth sense, the (1999)']
explain = als_model.explain(user, csr_data, itemid=once)

In [46]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('the greatest showman', 0.0007911399508540435),
 ('once', 0.000786743858906502),
 ('the lord of the rings', 0.0007756582526081869),
 ('shine', 0.0006895841970327899),
 ('farinelli', 0.0006611503177752383)]

#### 루브릭
- CSR matrix가 정상적으로 만들어졌다. 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.
- MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.
- 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다. MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.

## 회고

- 분석하기 에서 groupby 실행 하는데 4번 실패 하였다. 추후 스터디에 참여하여 복습할 예정이다. 
- 정상적으로  인덱싱이 성공할 경우 데이터 컬럼내 값을 user_id & title 숫자로 변경되는것이 신기 하였다.
- 영화명이 긴 경우 title_to_idx에는 영화명 그대로를,  als_model.item_factors[the_greatest_showman] 사이에는 언더바를 붙여야 오류가 나지 않았다.
- 추천받은 영화 리스트중 내가 작성하지 않은 5개외의 영화명을 입력함에도 내가 입력한 5개 영화와 동일해서 감탄 하였다. 
- 추천 받은 영화 리스트가 최신이 아니다 보니 나의 최근 변화된 영화 선호도를 확인하지 못한것이 좀 아쉬웠다.